In [ ]:
import sys; sys.path.append('..')
from osp import *

In [ ]:
get_current_feat_weights()

In [ ]:
for id,docstr in STASH_SLICES_NLP.items():
    break
# doc=stanza.Document.from_serialized(docstr)
# sent = random.choice(doc.sentences)
txt = """
These words, it seems to me, give us a particular picture of the
essence of human language. It is this: the individual words in language
name objects—sentences are combinations of such names.——In this
picture of language we find the roots of the following idea: Every word
has a meaning. This meaning is correlated with the word. It is the
object for which the word stands.
"""

doc = get_nlp_doc(txt)
sent = doc.sentences[0]

In [ ]:
def is_leaf(tree):
    """Check if a tree node is a leaf (preterminal) node."""
    return not isinstance(tree[0], nltk.Tree) if len(tree) > 0 else False


def get_leaf_ids(tree):
    print(tree)
    return [word.id for word in tree.leaves()]

def iter_preterms(tree):
    for t in tree.subtrees():
        if is_leaf(t):
            yield t

def tokenize_clauses(tree): # -> return both whether "IC" or "DC"
    clause_num = 0
    clause_type = None
    for t in tree.subtrees():
        ttxt = ' '.join(t.leaves())
        if ttxt and ttxt[0].isalpha() and ((not is_in_sbar(t) and t.label() == 'S') or t.label() == 'SBAR'):
            clause_num += 1
            clause_type = 'DC' if is_in_sbar(t) else 'IC'

        t._clause_num = clause_num
        t._clause_type = clause_type

    # o = defaultdict(list)
    o = []
    for preterm_i,preterm in enumerate(iter_preterms(tree)):
        d = {
            'clause_num': preterm._clause_num,
            'clause_type': preterm._clause_type,
            'word_num': preterm_i+1,
            'word': ' '.join(preterm.leaves())
        }
        o.append(d)
    
    odf = pd.DataFrame(o)
    odf['clause_num'] = odf['clause_num'].rank(method='dense', ascending=True).apply(int)
    return odf
    

In [ ]:
sent = get_nlp_doc("The trouble about progress is that it always looks much greater than it really is.").sentences[0]
# # sent

In [ ]:
def get_clauses(sent):
    if isinstance(sent, str):
        sent = get_nlp_doc(sent).sentences[0]
    tree = get_sent_tree(sent)
    odf = tokenize_clauses(tree)
    assert len(odf) == len(sent.words)
    odf['pos'] = [w.xpos for w in sent.words]
    odf['deprel'] = [w.deprel for w in sent.words]
    odf['head'] = [w.head for w in sent.words]
    return odf.set_index(['clause_num','clause_type','word_num'])

In [ ]:
df = get_clauses(sent)

In [ ]:
import matplotlib.pyplot as plt
import matplotlib.colors as mcolors

def get_color(
    feat,
    color_by='weight',
    df_feats=None,
    vcenter=0.0,  # The "clamping" point for the neutral color
    vmin=None,
    vmax=None,
    cmap_name='RdBu' # Red (low) to White to Blue (high)
):
    if df_feats is None:
        df_feats = get_current_feat_weights()
    
    val = df_feats.loc[feat, color_by] if feat in df_feats.index else 0
    
    # 1. Use provided bounds or calculate from data
    if vmin is None:
        vmin = df_feats[color_by].min()
    if vmax is None:
        vmax = df_feats[color_by].max()

    # 2. Logic to handle the center point clamping
    # TwoSlopeNorm requires vmin < vcenter < vmax
    if vmin < vcenter < vmax:
        norm = mcolors.TwoSlopeNorm(vmin=vmin, vcenter=vcenter, vmax=vmax)
    else:
        # Fallback if the range is entirely on one side of vcenter
        norm = mcolors.Normalize(vmin=vmin, vmax=vmax)
    
    # 3. Get the color
    print(feat,val,vmin,vmax)
    cmap = plt.get_cmap(cmap_name)
    rgba = cmap(norm(val))
    return mcolors.to_hex(rgba)

In [ ]:
get_color('pos_NNPS',color_by='weight')

In [ ]:
def get_manual_displacy_data(clause_df):
    clause_data = {
        "words": [{"text": text, "tag": pos} for text,pos in zip(clause_df['word'], clause_df['pos'])],
        "arcs": [
            {
                "start": row['word_num']-1,
                "end": row['head']-1,
                "label": row['deprel'],
                "dir": "left" if row['word_num'] > row['head'] else "right"
            }
            for _, row in clause_df.reset_index().iterrows()
            if row['word_num'] < row['head']
        ]
    }
    return clause_data

In [ ]:
clause_data = get_manual_displacy_data(df)

In [ ]:
# for clause, clause_df in df.groupby('clause_num'):
#     clause_data = get_manual_displacy_data(clause_df)
#     displacy.render(clause_data, style="dep", manual=True)
#     # x.render_to_file(f"clause_{clause}.svg")

In [ ]:
# !pip install beautifulsoup4
from bs4 import BeautifulSoup

In [ ]:
color_by = 'weight'

from IPython.display import HTML, display
from spacy import displacy
html_output = displacy.render(clause_data, style="dep", manual=True,options={'compact': False},jupyter=False)

# Parse the SVG
soup = BeautifulSoup(html_output, 'html.parser')

tag_elements = soup.find_all('tspan', class_='displacy-tag')
for idx, tag_elem in enumerate(tag_elements):
    color = get_color(f'pos_{tag_elem.text}', color_by=color_by)
    # Use a thick stroke as a background, and set paint-order to draw stroke behind fill
    tag_elem['style'] = f"paint-order: stroke fill; stroke: {color}; stroke-width: 1em; stroke-linecap: round; stroke-linejoin: round;"
    tag_elem['fill'] = 'black'  # Keep the text itself black for readability

deprel_elements = soup.find_all('textpath')
for idx, deprel_elem in enumerate(deprel_elements):
    color = get_color(f'deprel_{deprel_elem.text}', color_by=color_by)
    # Apply the same background highlight effect to dependency labels
    deprel_elem['style'] = f"paint-order: stroke fill; stroke: {color}; stroke-width: .3em; font-size: 1.1em;"
    deprel_elem['fill'] = 'black'

# # Combine and display
HTML(str(soup))
# # deprel_elements

In [ ]:
# print(html_output )

In [ ]:
stopxx

In [ ]:
displacy.serve(clause_data, style="dep", manual=True, port=5001)

In [ ]:
# Format your Stanza data into this specific dictionary structure
manual_data = {
    "words": [
        {"text": "Barack", "tag": "PROPN"},
        {"text": "Obama", "tag": "PROPN"},
        {"text": "was", "tag": "AUX"},
        {"text": "born", "tag": "VERB"}
    ],
    "arcs": [
        {"start": 0, "end": 1, "label": "flat", "dir": "left"},
        {"start": 1, "end": 3, "label": "nsubjpass", "dir": "left"},
        {"start": 2, "end": 3, "label": "auxpass", "dir": "left"}
    ]
}

displacy.render(manual_data, style="dep", manual=True)

In [ ]:
claus

In [ ]:
def get_displacy_data_from_sent(sent):
    words = [{"text": word.text, "tag": word.xpos} for word in sent.words]
    arcs = []
    
    for word in sent.words:
        # Skip root dependencies (head == 0)
        if word.head == 0:
            continue
        
        # Convert from 1-based (Stanza) to 0-based (displaCy) indexing
        start = word.head - 1  # head position (0-based)
        end = word.id - 1      # current word position (0-based)
        
        # Skip self-loops (word pointing to itself)
        if start == end:
            continue
        
        # Determine direction
        if start < end:
            direction = "left"
        else:
            direction = "right"
            # Swap start and end for right-pointing arcs
            start, end = end, start
        
        arcs.append({
            "start": start,
            "end": end,
            "label": word.deprel,
            "dir": direction
        })
    
    return {"words": words, "arcs": arcs}

data = get_displacy_data_from_sent(sent)

In [ ]:
displacy.render(data, style="dep", manual=True)